In [4]:
import argparse
import os
import sys
import numpy as np
from textacy.datasets.supreme_court import SupremeCourt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import pandas as pd

In [2]:
print('Processing text dataset')

sc = SupremeCourt()
print(sc.info)

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
issue_codes.sort()
issue_codes = [str(ic) for ic in issue_codes]

labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))

for record in sc.records():
    if record[1]['issue'] == None: # some cases have None as an issue
        labels.append(labels_index['-1'])
    else:
        labels.append(labels_index[record[1]['issue'][:-4]])
    texts.append(record[0])

print('Found %s texts.' % len(texts))
print('Found %s labels.' % len(labels_index))

Processing text dataset
{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
Found 8419 texts.
Found 15 labels.


In [3]:
for record in sc.records():
    print(record[1:])
    break

({'issue': '80180', 'issue_area': 8, 'n_min_votes': 1, 'case_name': 'HALLIBURTON OIL WELL CEMENTING CO. v. WALKER et al., DOING BUSINESS AS DEPTHOGRAPH CO.', 'maj_opinion_author': 78, 'decision_date': '1946-11-18', 'decision_direction': 'liberal', 'n_maj_votes': 8, 'us_cite_id': '329 U.S. 1', 'argument_date': '1946-01-09'},)


In [27]:
# read the scotus_c_global_table_file.tsv
file_path = '/misc/grice1/yijun/SCOTUS-Embedding/data/'
file_name = 'scotus_c_global_table_file.tsv'
citation_global_file = pd.read_csv(file_path + file_name, delimiter='\t', index_col=False, encoding='utf-8')
citation_global_file

,global_id,id1,lookup_key,entry_type,party1_id,party2_id,party1,party2,party1_short,party2_short,case_name,standard_reporter,volume,page_number
0,0.001,98802,rrcoal#1,party_key,None,None,PENNSYLVANIA RR,SONMAN COAL CO,RR,COAL,None,Unknown,Unknown,Unknown
1,1.001,109350,operatorskleppe#1,party_key,None,None,NATIONAL INDEPENDENT COAL OPERATORS ASSN,KLEPPE,OPERATORS,KLEPPE,None,Unknown,Unknown,Unknown
2,10.001,2521197,universitycity#1,party_key,None,None,BOB JONES UNIVERSITY INC,CITY OF GREENVILLE,UNIVERSITY,CITY,None,Unknown,Unknown,Unknown
3,100.001,94240,whitehorn#1,party_key,None,None,WHITE,VAN HORN,WHITE,HORN,None,Unknown,Unknown,Unknown
4,100.002,94240,u.s._159_3,standard_key,None,None,None,None,None,None,None,U.S.,159,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613362,99995.001,Dummy_File_36145,None,case_X_vs_Y,None,None,Richmond Concrete Products Co.,Ward,Products,Ward,None,None,None,None
613363,99996.001,Dummy_File_36146,s.e.2d_95_679,standard_key,None,None,None,None,None,None,None,S.E.2D,95,679
613364,99997.001,Dummy_File_36147,ga._181_274,standard_key,None,None,None,None,None,None,None,GA.,181,274
613365,99998.001,Dummy_File_36148,s.e._182_19,standard_key,None,None,None,None,None,None,None,S.E.,182,19


In [30]:
def search_global_id(global_id):
    return citation_global_file[citation_global_file['global_id'].isin([global_id])]

search_global_id('1.001')

,global_id,id1,lookup_key,entry_type,party1_id,party2_id,party1,party2,party1_short,party2_short,case_name,standard_reporter,volume,page_number
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
613363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
613364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
613365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
citation_global_file.loc[citation_global_file['global_id'].isin(["99999.001"])]

,global_id,id1,lookup_key,entry_type,party1_id,party2_id,party1,party2,party1_short,party2_short,case_name,standard_reporter,volume,page_number


In [38]:
citation_global_file[-3:]

,global_id,id1,lookup_key,entry_type,party1_id,party2_id,party1,party2,party1_short,party2_short,case_name,standard_reporter,volume,page_number
613364,99997.001,Dummy_File_36147,ga._181_274,standard_key,None,None,None,None,None,None,None,GA.,181,274
613365,99998.001,Dummy_File_36148,s.e._182_19,standard_key,None,None,None,None,None,None,None,S.E.,182,19
613366,99999.001,Dummy_File_36149,ga._209_522,standard_key,None,None,None,None,None,None,None,GA.,209,522


In [46]:
citation_global_file['global_id'].isin([99999.001])

0         False
1         False
2         False
3         False
4         False
          ...  
613362    False
613363    False
613364    False
613365    False
613366    False
Name: global_id, Length: 613367, dtype: bool

In [55]:
for _, row in citation_global_file.iterrows():
#     print(row['global_id'])
#     break
#     print(row)
    if row['global_id'] == 99999.001:
        print(True)

KeyboardInterrupt: 

In [ ]:
for index, row in citation_global_file.iterrows():
    if type(row['global_id']) != float:
        print(type(row['global_id']))
        print(index)
#     break